# SQL проект - анализ базы данных книг сервиса подписки
**Описание проекта**

Коронавирус застал мир врасплох, изменив привычный порядок вещей. В свободное время жители городов больше не выходят на улицу, не посещают кафе и торговые центры. Зато стало больше времени для книг. Это заметили стартаперы — и бросились создавать приложения для тех, кто любит читать.


Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке.

**Цель исследования**
Проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

**Задания**
1. Посчитайте, сколько книг вышло после 1 января 2000 года;
2. Для каждой книги посчитайте количество обзоров и среднюю оценку;
3. Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
4. Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
5. Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

<a id='section_id4'></a>
**Оглавление**
1. [Импорт библиотек и подключение к СУБД](#section_id1)
2. [Анализ базы данных](#section_id2)
3. [Результаты исследования](#section_id3)

<a id='section_id1'></a>
## Импорт библиотек и подключение к СУБД

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

# чтобы выполнить SQL-запрос, используем Pandas
query = '''SELECT * FROM books LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [2]:
query = '''SELECT * FROM authors LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [3]:
query = '''SELECT * FROM ratings LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [4]:
query = '''SELECT * FROM reviews LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [5]:
query = '''SELECT * FROM publishers LIMIT 5'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


**Выводы:** получено общее представление о содержании всех таблиц, выведены первые 5 строк каждой из них.

[Оглавление](#section_id4)

<a id='section_id2'></a>
## Анализ базы данных

### Посчитайте, сколько книг вышло после 1 января 2000 года

In [6]:
query = '''SELECT COUNT(book_id)
FROM books
WHERE publication_date > '2000-01-01' '''

pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,819


**Выводы:** с 1 января 2000 года было выпущено 819 книг.

### Для каждой книги посчитайте количество обзоров и среднюю оценку;

In [7]:
query = '''SELECT b.title, COUNT(rw.review_id) cnt_reviews, ROUND(AVG(rt.rating), 2) avg_rating
FROM books b
LEFT JOIN ratings rt ON b.book_id = rt.book_id
LEFT JOIN reviews rw ON b.book_id = rw.book_id
GROUP BY 1
ORDER BY 2 DESC
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,title,cnt_reviews,avg_rating
0,Twilight (Twilight #1),1120,3.66
1,The Hobbit or There and Back Again,528,4.13
2,The Catcher in the Rye,516,3.83
3,Harry Potter and the Prisoner of Azkaban (Harr...,492,4.41
4,Harry Potter and the Chamber of Secrets (Harry...,480,4.29
...,...,...,...
994,The Natural Way to Draw,0,3.00
995,Essential Tales and Poems,0,4.00
996,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
997,Leonardo's Notebooks,0,4.00


**Выводы:** со значительным отрывом по количеству обзоров лидирует книга Сумерки.

In [8]:
query = '''SELECT SUM(cnt_reviews)
FROM (
SELECT b.title, COUNT(rw.review_id) cnt_reviews, ROUND(AVG(rt.rating), 2) avg_rating
FROM books b
LEFT JOIN ratings rt ON b.book_id = rt.book_id
LEFT JOIN reviews rw ON b.book_id = rw.book_id
GROUP BY 1
ORDER BY 2 DESC) t1
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,sum
0,26167.0


In [9]:
query = '''SELECT COUNT(review_id)
FROM reviews
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,count
0,2793


In [10]:
query = '''WITH rw AS (SELECT book_id, COUNT(review_id) cnt_reviews FROM reviews GROUP BY 1),
rt AS (SELECT book_id, ROUND(AVG(rating), 2) avg_rating FROM ratings  GROUP BY 1)

SELECT b.book_id, b.title, COALESCE(rw.cnt_reviews, 0) cnt_reviews, rt.avg_rating
FROM books b
LEFT JOIN rw ON b.book_id = rw.book_id
LEFT JOIN rt ON b.book_id = rt.book_id
ORDER BY 3 DESC
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,book_id,title,cnt_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,854,The Road,6,3.77
2,627,The Alchemist,6,3.79
3,695,The Curious Incident of the Dog in the Night-Time,6,4.08
4,733,The Giver (The Giver #1),6,3.75
...,...,...,...,...
995,672,The Cat in the Hat and Other Dr. Seuss Favorites,0,5.00
996,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


In [11]:
query = '''WITH rw AS (SELECT book_id, COUNT(review_id) cnt_reviews FROM reviews GROUP BY 1),
rt AS (SELECT book_id, ROUND(AVG(rating), 2) avg_rating FROM ratings  GROUP BY 1)

SELECT SUM(cnt_reviews) FROM 
(SELECT b.book_id, b.title, COALESCE(rw.cnt_reviews, 0) cnt_reviews, rt.avg_rating
FROM books b
LEFT JOIN rw ON b.book_id = rw.book_id
LEFT JOIN rt ON b.book_id = rt.book_id
ORDER BY 3 DESC) t1
'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,sum
0,2793.0


**Выводы:** по количеству обзоров лидирует книга Сумерки.

### Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры

In [12]:
query = '''WITH t1 AS (SELECT publisher_id, COUNT(book_id) books_cnt
                    FROM books
                    WHERE num_pages > 50
                    GROUP BY 1
                    ORDER BY 2 DESC
                    LIMIT 1)
SELECT publisher, books_cnt
FROM publishers p
LEFT JOIN t1 ON p.publisher_id = t1.publisher_id
WHERE p.publisher_id IN (SELECT publisher_id
                    FROM t1)
ORDER BY books_cnt DESC'''

pd.io.sql.read_sql(sql=text(query), con = engine)

,publisher,books_cnt
0,Penguin Books,42


**Выводы:** больше всего книг было выпущено издательством Penguin Books - 42.

### Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками

In [13]:
query = '''SELECT author
FROM authors
WHERE author_id = (
SELECT author_id
FROM books
WHERE book_id = (
SELECT book_id FROM ratings GROUP BY 1 HAVING COUNT(rating) > 50 ORDER BY ROUND(AVG(rating), 2) DESC LIMIT 1))
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,author
0,J.K. Rowling/Mary GrandPré


**Выводы:** автором с самой высокой средней оценкой книг явялется Джон Роулинг.

### Посчитайте среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [14]:
query = '''WITH top_users AS (SELECT username FROM ratings GROUP BY username HAVING COUNT(rating_id) > 50)
SELECT ROUND(COUNT(review_id) / COUNT(DISTINCT username)::decimal, 2) avg_reviews
FROM reviews
WHERE username IN (SELECT * FROM top_users)
'''
pd.io.sql.read_sql(sql=text(query), con = engine)

,avg_reviews
0,24.33


**Выводы:** пользователи, которые поставили более 50 оценок, написали в среднем 24 отзыва.

[Оглавление](#section_id4)

<a id='section_id3'></a>
## Результаты исследования

1. С 1 января 2000 года было выпущено 819 книг.
2. По количеству обзоров лидирует книга Сумерки.
3. Больше всего книг было выпущено издательством Penguin Books - 42.
4. Автором с самой высокой средней оценкой книг явялется Джон Роулинг.
5. Пользователи, которые поставили более 50 оценок, написали в среднем 24 отзыва.

[Оглавление](#section_id4)